In [ ]:
!pip install efficientnet_pytorch codecarbon


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from codecarbon import EmissionsTracker  # For tracking CO₂ emissions
import time

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

num_epochs = 20  # Change if needed
batch_size = 32  # Adjust for Kaggle GPU memory
learning_rate = 1e-3  # Initial learning rate
image_size = 224  # EfficientNet requires 224x224 input images



In [ ]:
# Define Transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Resize CIFAR-100 images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])  # CIFAR-100 normalization
])

# Load CIFAR-100 Dataset
train_dataset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print("Dataset Loaded Successfully ")

# Load EfficientNet-B4 with adjusted output layer for 100 classes
model = EfficientNet.from_pretrained("efficientnet-b4", num_classes=100)
model.to(device)
print("Model Loaded ")


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler to adjust learning rate when validation loss plateaus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, verbose=True)


In [ ]:
tracker = EmissionsTracker(project_name="EfficientNet_CIFAR100_Kaggle")
tracker.start()
print("Tracking CO₂ Emissions... ")


In [ ]:
# Mixed Precision Training (AMP)
scaler = torch.cuda.amp.GradScaler()

def train_epoch(epoch, pbar):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        
        # Mixed precision training
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar with current batch progress
        pbar.update(1)
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def evaluate():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    loss = running_loss / total
    acc = 100. * correct / total
    return loss, acc


In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar

start_time = time.time()  # Track total training time

for epoch in range(num_epochs):
    epoch_start = time.time()  # Track epoch start time
    
    # Create a tqdm progress bar for training
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        train_loss, train_acc = train_epoch(epoch, pbar)  # Pass progress bar to function
    
    val_loss, val_acc = evaluate()  # Validation after each epoch
    
    epoch_time = time.time() - epoch_start  # Calculate epoch duration
    
    # Adjust Learning Rate
    scheduler.step(val_loss)
    
    print(f"\nEpoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | "
          f"Epoch Time: {epoch_time:.2f} sec")
    
total_time = time.time() - start_time  # Total training duration
print(f"\nTotal Training Time: {total_time/60:.2f} minutes")



In [ ]:
# Stop CodeCarbon tracker and report total CO₂ emissions
emissions = tracker.stop()
print(f"Total CO₂ Emissions: {emissions:.6f} kg")


In [ ]:
# Save trained model for future inference
torch.save(model.state_dict(), "efficientnet_cifar100.pth")
print("Model Saved Successfully ")
